# CIFAR Conv Net

И так, в этом ноутбуке Вы сделаете превую в своей жизни сверточную сеть! На сложном датасете. Cкачайте его кстати, 

In [ ]:
!mkdir cifar10
!curl -o cifar-10-python.tar.gz https://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz
!tar -xvzf cifar-10-python.tar.gz -C cifar10

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.cluster.vq import whiten

%matplotlib inline

In [2]:
from cifar import load_CIFAR10
plt.rcParams['figure.figsize'] = (10.0, 8.0) 

cifar10_dir = './cifar10/cifar-10-batches-py'
X_train, y_train, X_test, y_test = load_CIFAR10(cifar10_dir)

In [3]:
X_train.shape

(50000, 3, 32, 32)

In [4]:
X_train = whiten(X_train)
X_test = whiten(X_test)

In [5]:
np.mean(X_train), np.std(X_train)

(1.911570213067062, 1.0102662581806696)

In [6]:
E = np.mean(X_train)
S = np.std(X_train)

In [8]:
# classes = ['plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
# num_classes = len(classes)
# samples_per_class = 7
# for y, cls in enumerate(classes):
#     idxs = np.flatnonzero(y_train == y)
#     idxs = np.random.choice(idxs, samples_per_class, replace=False)
#     for i, idx in enumerate(idxs):
#         plt_idx = i * num_classes + y + 1
#         plt.subplot(samples_per_class, num_classes, plt_idx)
#         plt.imshow(X_train[idx].astype('uint8').transpose(1, 2, 0))
#         plt.axis('off')
#         if i == 0:
#             plt.title(cls)
# plt.show()

<h1 align="center">First of all -- Checking Questions</h1> 

**Вопрос 1**: Чем отличаются современные сверточные сети от сетей 5 летней давности?

<Ответ>

**Вопрос 2**: Какие неприятности могут возникнуть во время обучения современных нейросетей?

<Ответ>


**Вопрос 3**: У вас есть очень маленький датасет из 100 картинок, классификация, но вы очень хотите использовать нейросеть, какие неприятности вас ждут и как их решить? что делать если первый вариант  решения не заработает?

Сделаю аугментацию данных(например добавлю повернутые/отраженные/инвертированные картинки)

**Вопрос 4**: Как сделать стайл трансфер для музыки? oO

Это рассказывали на 1 доп семинаре по машинному обучению. Можно воспользоваться той же идеей, что и при стайл трансфере изображний, только накладывать спектр одного трека на другой, рассматривая спектры как изображения размера 1xt.

In [9]:
import theano

WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

Using gpu device 0: Tesla K80 (CNMeM is enabled with initial size: 95.0% of memory, cuDNN 5110)


In [10]:
import lasagne
from theano import tensor as T
from lasagne.nonlinearities import *
from theano import tensor as T
from lasagne.nonlinearities import softmax
from lasagne import init

from lasagne.layers import InputLayer, DropoutLayer, FlattenLayer, DenseLayer, batch_norm
from lasagne.layers import ElemwiseMergeLayer, ElemwiseSumLayer, NonlinearityLayer
from lasagne.layers.dnn import Conv2DDNNLayer as ConvLayer
from lasagne.layers import Pool2DLayer as PoolLayer


input_X = T.tensor4("X")
target_y = T.vector("target Y integer",dtype='int32')

### Соберите нейронку: 
- Many times x (Conv+Pool)
- Many small convolutions like 3x3
- Batch Norm 
- Residual Connection
- Data Augmentation 
- Learning rate Schedule 
- ...

### Для вдохновения 
- http://torch.ch/blog/2015/07/30/cifar.html
- https://github.com/szagoruyko/wide-residual-networks 

### Самое интересное
- Для сдачи задания нужно набрать на точность тесте > **92.5**% (это займет много времени, торопитесь :) )
- Для получения бонусных баллов > **95.0**%
- Будет очень хорошо если вы придумаете свою архитектуру или сможете обучить что-то из вышеперечисленного :)
- А для обучения всего этого добра вам будет куда удобнее использовать GPU на Amazon 
    - Инструкция https://github.com/persiyanov/ml-mipt/tree/master/amazon-howto 
    - Вам помогут tmux, CuDNN, ssh tunnel, nvidia-smi, ... 
    - Have fun :)

In [11]:
# так себе ~50-70%
def build_model_1(X):
    input_layer = lasagne.layers.InputLayer(shape=(None,3, 32, 32), input_var=X)

    # nnet = lasagne.layers.BatchNormLayer(input_layer, gamma=np.ones(3), epsilon=1e-9)
    nnet = lasagne.layers.Conv2DLayer(input_layer, num_filters=10, filter_size=(8, 8),
                                      pad='valid', W=init.GlorotNormal(), nonlinearity=sigmoid)
    nnet = lasagne.layers.MaxPool2DLayer(nnet, pool_size=(2, 2))
    nnet = lasagne.layers.Conv2DLayer(input_layer, num_filters=40, filter_size=(8, 8),
                                      pad='valid', W=init.GlorotNormal(), nonlinearity=sigmoid)
    nnet = lasagne.layers.MaxPool2DLayer(nnet, pool_size=(2, 2))
    nnet = lasagne.layers.Conv2DLayer(input_layer, num_filters=100, filter_size=(8, 8),
                                      pad='valid', W=init.GlorotNormal(), nonlinearity=sigmoid)
    nnet = lasagne.layers.MaxPool2DLayer(nnet, pool_size=(2, 2))
    nnet = lasagne.layers.Conv2DLayer(nnet, num_filters=10*10, filter_size=(3, 3), 
                                      pad='valid', W=init.GlorotNormal(), nonlinearity=sigmoid)
    nnet = lasagne.layers.Conv2DLayer(nnet, num_filters=10*10, filter_size=(3, 3), 
                                      pad='valid', W=init.GlorotNormal(), nonlinearity=sigmoid)
    nnet = lasagne.layers.Conv2DLayer(nnet, num_filters=10*10, filter_size=(3, 3), 
                                      pad='valid', W=init.GlorotNormal(), nonlinearity=sigmoid)
    nnet = lasagne.layers.BatchNormLayer(nnet)
    nnet = lasagne.layers.DenseLayer(nnet, num_units=100,nonlinearity=rectify)
    nnet = lasagne.layers.DropoutLayer(nnet,p=0.42)
    nnet = lasagne.layers.DenseLayer(nnet,num_units=64, nonlinearity=rectify)

    return lasagne.layers.DenseLayer(nnet,num_units = 10, nonlinearity=softmax)

In [12]:
# хорошая модель ~70-75%
def build_model_2(X):
    input_layer = lasagne.layers.InputLayer(shape=(None,3, 32, 32), input_var=X)

    nnet = lasagne.layers.Conv2DLayer(input_layer, num_filters=64, filter_size=(5, 5), nonlinearity=sigmoid, pad='valid', W=init.GlorotNormal())
    nnet = lasagne.layers.DropoutLayer(nnet,p=0.23)
    nnet = lasagne.layers.Conv2DLayer(nnet, num_filters=64, filter_size=(3, 3), nonlinearity=sigmoid, pad='valid', W=init.GlorotNormal())
    nnet = lasagne.layers.MaxPool2DLayer(nnet, pool_size=(2, 2))
    nnet = lasagne.layers.DropoutLayer(nnet,p=0.23)
    nnet = lasagne.layers.Conv2DLayer(nnet, num_filters=64, filter_size=(3, 3), nonlinearity=sigmoid, pad='valid', W=init.GlorotNormal())
    nnet = lasagne.layers.MaxPool2DLayer(nnet, pool_size=(2, 2))
    nnet = lasagne.layers.DropoutLayer(nnet,p=0.23)
    nnet = lasagne.layers.Conv2DLayer(nnet, num_filters=64, filter_size=(3, 3), nonlinearity=sigmoid, 
                                      pad='valid', W=init.GlorotNormal())
    nnet = lasagne.layers.MaxPool2DLayer(nnet, pool_size=(2, 2))
    nnet = lasagne.layers.DenseLayer(nnet,num_units=64, nonlinearity=sigmoid)

    return lasagne.layers.DenseLayer(nnet,num_units = 10, nonlinearity=softmax)

In [13]:
# тоже ничего ~70%
def build_model_3(X):
    net = {}
    net['input'] = InputLayer(shape=(None, 3, 32, 32), input_var=X)
    net['conv1'] = ConvLayer(net['input'],
                             num_filters=192,
                             filter_size=5,
                             pad=2,
                             flip_filters=False)
    net['cccp1'] = ConvLayer(
        net['conv1'], num_filters=160, filter_size=1, flip_filters=False)
    net['cccp2'] = ConvLayer(
        net['cccp1'], num_filters=96, filter_size=1, flip_filters=False)
    net['pool1'] = PoolLayer(net['cccp2'],
                             pool_size=3,
                             stride=2,
                             mode='max',
                             ignore_border=False)
    net['drop3'] = DropoutLayer(net['pool1'], p=0.5)
    net['conv2'] = ConvLayer(net['drop3'],
                             num_filters=192,
                             filter_size=5,
                             pad=2,
                             flip_filters=False)
    net['cccp3'] = ConvLayer(
        net['conv2'], num_filters=192, filter_size=1, flip_filters=False)
    net['cccp4'] = ConvLayer(
        net['cccp3'], num_filters=192, filter_size=1, flip_filters=False)
    net['pool2'] = PoolLayer(net['cccp4'],
                             pool_size=3,
                             stride=2,
                             mode='average_exc_pad',
                             ignore_border=False)
    net['drop6'] = DropoutLayer(net['pool2'], p=0.5)
    net['conv3'] = ConvLayer(net['drop6'],
                             num_filters=192,
                             filter_size=3,
                             pad=1,
                             flip_filters=False)
    net['cccp5'] = ConvLayer(
        net['conv3'], num_filters=192, filter_size=1, flip_filters=False)
    net['cccp6'] = ConvLayer(
        net['cccp5'], num_filters=10, filter_size=1, flip_filters=False)
    net['pool3'] = PoolLayer(net['cccp6'],
                             pool_size=8,
                             mode='average_exc_pad',
                             ignore_border=False)
#     net['output'] = FlattenLayer(net['pool3'])
    net['output'] = DenseLayer(net['pool3'], num_units=10, nonlinearity=softmax)

    return net['output']

In [13]:
# одна из лучших ~84%
def x2Conv3x3Block(net,n,m):
    net = ConvLayer(net, num_filters=n, filter_size=3, pad='same', nonlinearity=rectify, W=init.GlorotNormal())
    net = lasagne.layers.BatchNormLayer(net)
    net = ConvLayer(net, num_filters=m, filter_size=3, pad='same', nonlinearity=rectify, W=init.GlorotNormal())
    net = PoolLayer(net, pool_size=(2,2), stride=(2,2), mode='max')
    return net

def build_model_4(X):
    net = InputLayer(shape=(None, 3, 32, 32), input_var=X)
    net = x2Conv3x3Block(net, 64, 128)
    net = x2Conv3x3Block(net, 96, 192)
    net = x2Conv3x3Block(net, 128, 256)
    net = ConvLayer(net, num_filters=160, filter_size=3, pad='same', nonlinearity=rectify, W=init.GlorotNormal())
    net = ConvLayer(net, num_filters=320, filter_size=3, pad='same', W=init.GlorotNormal())
    net = PoolLayer(net, pool_size=(4,4),mode='average_exc_pad')
    net = DenseLayer(net, num_units=64, nonlinearity=sigmoid,W=init.GlorotNormal())
    net = DenseLayer(net, num_units=10, nonlinearity=softmax,W=init.GlorotNormal())
    return net

In [14]:
# еще немного лучше ~88%
def x2Conv3x3BlockWithRes(layer,n,m):
    net = layer
    if (m != layer.output_shape[1]):
        layer = ConvLayer(layer, m, filter_size=1, pad=0, nonlinearity=None, b=None)
    net = ConvLayer(net, num_filters=n, filter_size=3, pad='same', nonlinearity=rectify, W=init.GlorotNormal())
    net = batch_norm(net)
    net = ConvLayer(net, num_filters=m, filter_size=3, pad='same', nonlinearity=None, W=init.GlorotNormal())
    net = ElemwiseSumLayer([net, layer])
    net = NonlinearityLayer(net, nonlinearity=rectify)
    net = PoolLayer(net, pool_size=(2,2), stride=(2,2), mode='max')
    return net
    
def build_model_5(X):
    net = InputLayer(shape=(None, 3, 32, 32), input_var=X)
    net = x2Conv3x3BlockWithRes(net, 64, 128)
    net = x2Conv3x3BlockWithRes(net, 96, 192)
    net = x2Conv3x3BlockWithRes(net, 128, 256)
    net = ConvLayer(net, num_filters=160, filter_size=3, pad='same', nonlinearity=rectify, W=init.GlorotNormal())
    net = batch_norm(net)
    net = ConvLayer(net, num_filters=320, filter_size=3, pad='same', W=init.GlorotNormal())
    net = batch_norm(net)
    net = PoolLayer(net, pool_size=(4,4),mode='max')
    net = DenseLayer(net, num_units=4096, nonlinearity=sigmoid,W=init.GlorotNormal())
    net = DenseLayer(net, num_units=10, nonlinearity=softmax,W=init.GlorotNormal())
    return net

За основу взята сеть из статьи https://habrahabr.ru/post/309302/ (она не была предназначена специально для этой задачи), которая давала около 80%. Далее сеть была сильно переработана: добавлено больше batch_norm слоев, а также добавлен слой, пробрасывающий данные между слоями сверток. При обучении вероятностно отражаются и обрезаются объекты из батча. Лернинг рейт тоже меняется по-умному. В итоге получается ~ 88%

In [15]:
net = build_model_5(input_X)

In [16]:
y_predicted = lasagne.layers.get_output(net)

In [17]:
all_weights = lasagne.layers.get_all_params(net)

In [20]:
loss = lasagne.objectives.categorical_crossentropy(y_predicted, target_y).mean()
accuracy = lasagne.objectives.categorical_accuracy(y_predicted, target_y).mean()

In [21]:
params = lasagne.layers.get_all_params(net, trainable=True)
print params

[W, beta, gamma, W, b, W, W, beta, gamma, W, b, W, W, beta, gamma, W, b, W, W, beta, gamma, W, beta, gamma, W, b, W, b]


In [22]:
train_fun = theano.function([input_X,target_y],[loss, accuracy], updates=lasagne.updates.adam(loss, params), 
                            allow_input_downcast=True)
accuracy_fun = theano.function([input_X,target_y],accuracy, allow_input_downcast=True)

In [23]:
from scipy import stats as sps

### Вот и всё, пошли её учить

In [24]:
def _flip(X):
    return np.array([ np.flip(x,2) if sps.bernoulli.rvs(p=0.3) else x for x in X])

def _clip(X):
    return np.array([ np.clip(x,E-2*S,E+2*S) if sps.bernoulli.rvs(p=0.3) else x for x in X])

def iterate_minibatches(inputs, targets, batchsize, shuffle=False):
    assert len(inputs) == len(targets)
    if shuffle:
        indices = np.arange(len(inputs))
        np.random.shuffle(indices)
    for start_idx in range(0, len(inputs) - batchsize + 1, batchsize):
        if shuffle:
            excerpt = indices[start_idx:start_idx + batchsize]
        else:
            excerpt = slice(start_idx, start_idx + batchsize)
        yield inputs[excerpt], targets[excerpt]
        
def custom_rate(n):
    return 0.001 * 0.1 ** (n/10)

In [25]:
print(theano.sandbox.cuda.dnn.version())

(5110, 5110)


In [23]:
# import sys
# sys.stdout = open('/dev/stdout', 'w')

# Процесс обучения

In [28]:
import time

num_epochs = 50 #количество проходов по данным

batch_size = 50 #размер мини-батча

for epoch in range(num_epochs):
    # In each epoch, we do a full pass over the training data:
    train_err = 0
    train_acc = 0
    train_batches = 0
    start_time = time.time()
    if epoch % 10 == 0:
        train_fun = theano.function([input_X,target_y],[loss, accuracy], 
                                updates=lasagne.updates.adam(loss, params,learning_rate= custom_rate(epoch)), 
                            allow_input_downcast=True)
    for batch in iterate_minibatches(X_train, y_train,batch_size,True):
        inputs, targets = batch
        inputs = _clip(_flip(inputs))
        train_err_batch, train_acc_batch= train_fun(inputs, targets)
        train_err += train_err_batch
        train_acc += train_acc_batch
        train_batches += 1

    # And a full pass over the validation data:
    val_acc = 0
    val_batches = 0
    for batch in iterate_minibatches(X_test, y_test, batch_size):
        inputs, targets = batch
        val_acc += accuracy_fun(inputs, targets)
        val_batches += 1

    # Then we print the results for this epoch:
    print("Epoch {} of {} took {:.3f}s".format(epoch + 1, num_epochs, time.time() - start_time))
    print("  training loss (in-iteration):\t\t{:.6f}".format(train_err / train_batches))
    print("  train accuracy:\t\t{:.2f} %".format(train_acc / train_batches * 100))
    print("  validation accuracy:\t\t{:.2f} %".format(val_acc / val_batches * 100))

Epoch 1 of 50 took 108.822s
  training loss (in-iteration):		0.660699
  train accuracy:		77.11 %
  validation accuracy:		77.65 %
Epoch 2 of 50 took 83.962s
  training loss (in-iteration):		0.517738
  train accuracy:		82.27 %
  validation accuracy:		81.82 %
Epoch 3 of 50 took 84.005s
  training loss (in-iteration):		0.434702
  train accuracy:		85.10 %
  validation accuracy:		82.84 %
Epoch 4 of 50 took 84.019s
  training loss (in-iteration):		0.371781
  train accuracy:		87.32 %
  validation accuracy:		83.09 %
Epoch 5 of 50 took 84.077s
  training loss (in-iteration):		0.314091
  train accuracy:		89.24 %
  validation accuracy:		84.28 %
Epoch 6 of 50 took 84.008s
  training loss (in-iteration):		0.275054
  train accuracy:		90.69 %
  validation accuracy:		85.39 %
Epoch 7 of 50 took 83.999s
  training loss (in-iteration):		0.238816
  train accuracy:		91.87 %
  validation accuracy:		85.80 %
Epoch 8 of 50 took 84.015s
  training loss (in-iteration):		0.204405
  train accuracy:		93.01 %
  valid

In [29]:
test_acc = 0
test_batches = 0
for batch in iterate_minibatches(X_test, y_test, 500):
    inputs, targets = batch
    acc = accuracy_fun(inputs, targets)
    test_acc += acc
    test_batches += 1
print("Final results:")
print("  test accuracy:\t\t{:.2f} %".format(
    test_acc / test_batches * 100))

if test_acc / test_batches * 100 > 92.5:
    print "Achievement unlocked: колдун 80 уровня"
else:
    print "Нужно больше магии!"

Final results:
  test accuracy:		89.01 %
Нужно больше магии!


Ясно, что можно было легко получить больше, используя специальные готовые архитектуры преднанченные именно для этих данных, но это не очень интересно.

## Заполните форму

https://goo.gl/forms/EeadABISlVmdJqgr2 